In [1]:
# Vorstellung: MNIST-Daten!
# http://yann.lecun.com/exdb/mnist/
# FashionMNIST: https://github.com/zalandoresearch/fashion-mnist

import gzip
import numpy as np
from tensorflow.keras.utils import to_categorical

def open_images(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=16)\
            .reshape(-1, 28, 28)\
            .astype(np.float32)


def open_labels(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=8)
    
X_train = open_images("../data/fashion/train-images-idx3-ubyte.gz")
y_train = open_labels("../data/fashion/train-labels-idx1-ubyte.gz")

X_test = open_images("../data/fashion/t10k-images-idx3-ubyte.gz")
y_test = open_labels("../data/fashion/t10k-labels-idx1-ubyte.gz")

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

2025-10-01 12:24:43.321464: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-01 12:24:43.526908: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-01 12:24:55.015794: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


### Neuronales Netz (mit Softmax)

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
model = Sequential()

model.add(Dense(200, activation="sigmoid", input_shape=(784,)))
model.add(Dense(100, activation="sigmoid", input_shape=(784,)))
model.add(Dense(10, activation="softmax"))

# rmsprop, adam
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

/home/alex/anaconda3/envs/ml/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-10-01 12:25:05.243457: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
model.fit(
    X_train.reshape(60000, 784),
    y_train,
    epochs=100,
    batch_size=1000)

Epoch 1/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 88ms/step - accuracy: 0.6311 - loss: 1.3257
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 89ms/step - accuracy: 0.7675 - loss: 0.7095
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.7968 - loss: 0.5947
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 139ms/step - accuracy: 0.8146 - loss: 0.5435
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 148ms/step - accuracy: 0.8240 - loss: 0.5103
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 132ms/step - accuracy: 0.8256 - loss: 0.5026
Epoch 7/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 112ms/step - accuracy: 0.8245 - loss: 0.4972
Epoch 8/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 113ms/step - accuracy: 0.8262 - loss: 0.4908
Epoch 9/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.8268 - loss: 0.4861
Epoch 10/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 108ms/step - accuracy: 0.8254 - loss: 0.4888
Epoch 11/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 12s 130ms/step - accuracy: 0.8314 - loss: 0.4763
Epoch 12/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 6

### Es gilt: Das Netz passt sich an die Daten an.

- Testgenauigkeit nur minimal schlechter wie die Trainingsgenauigkeit: Das Modell darf vermutlich noch was komplexer sein 

- Testgenauigkeit sehr viel schlechter wie die Trainingsgenauigkeit: Das Modell ist zu komplex (das Modell hat sich zu stark an die Trainingsdaten angepasst)

- Beides schlecht: Du brauchst mehr Daten!

In [5]:
model.evaluate(X_train.reshape(-1, 784), y_train)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8525 - loss: 0.4049


[0.40494000911712646, 0.852483332157135]

In [6]:
model.evaluate(X_test.reshape(-1, 784), y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8355 - loss: 0.4523


[0.4523372948169708, 0.8355000019073486]